# Overview

This notebook demonstrates how to use the ```BatchProcessor``` class to processes lists of text or pandas dataframes with text columns efficiently in parallel using Azure OpenAI's API. The class provides a simple and efficient way to process large amounts of text data in parallel, with support for rate limiting and error handling.

It also shows how to use pydantic models to structure LLM output and perform type validation which is **highly** recommended since the primary purpose of these tools is to extract structured data from unstructured text.

## Notes
- Be aware of when you are asking the LLM to extract things that might not be present in the text. This will cause hallucinations.


In [1]:
# Imports
from utils.llm.batch_llm import BatchProcessor
from utils.llm.llm_config import LLMConfig, LLMConfigs

from pprint import pprint
from pydantic import BaseModel
import pandas as pd

# Processor Initialization

The processor takes four arguments:
1. model: The name of the Azure deployment to use. If none is specified, it looks for the AZURE_DEPLOYMENT_NAME environment variable.
2. max_tokens_per_minute: The maximum number of tokens to use per minute which is set for the deployment in Azure AI foundery. The default is 704,000.
3. safety_margin: A safety margin to apply to the token limit. This is needed because the actual token usage can vary from the estimated token usage. The default is 0.9 (90%).
4. max_concurrent: The maximum number of concurrent requests to make. The default is 50.


In [ ]:
# Example Initialization

config = LLMConfigs.openrouter(model="google/gemini-2.5-flash-lite")
processor = BatchProcessor(llm_config=config)

# Process Single

The ```process_single``` method takes a single text and an optional response model. The response model can be used to structure LLM output and perform type validation. 

In [3]:
# With response model, returns an instance of the model

class Book(BaseModel):
    title: str
    author: str

result = await processor.process_single("The Name of the Wind by Patrick Rothfuss", response_model=Book)
pprint(result)
pprint(type(result))


Book(title='The Name of the Wind', author='Patrick Rothfuss')
<class '__main__.Book'>


In [4]:
# Without response model, returns a string

result = await processor.process_single("Give the title and author of this book in json format: The Name of the Wind by Patrick Rothfuss")
print(result)
print(type(result))

```json
{
  "title": "The Name of the Wind",
  "author": "Patrick Rothfuss"
}
```
<class 'str'>


# Process Batch

The ```process_batch``` method takes a list of texts and an optional response model. When a response model is provided, the method returns a list of instances of the model that can be converted to JSON or other formats. 

Objects like this are often used in validation/serialization for APIs and when moving large volumes of data.

In [5]:
test_texts = [
    "To Kill a Mockingbird by Harper Lee (1960)",
    "The Great Gatsby Author: F. Scott Fitzgerald Publication Year: 1925",
    "Book: 1984 Writer: George Orwell Year: 1949",
    "Austen, Jane. Pride and Prejudice. 1813.",
    "One Hundred Years of Solitude - Gabriel García Márquez, published in 1967"
]

class Book(BaseModel):
    title: str
    author: str
    year: int

results = await processor.process_batch(test_texts, response_model=Book)
pprint(results)

[Book(title='To Kill a Mockingbird', author='Harper Lee', year=1960),
 Book(title='The Great Gatsby', author='F. Scott Fitzgerald', year=1925),
 Book(title='1984', author='George Orwell', year=1949),
 Book(title='Pride and Prejudice', author='Jane Austen', year=1813),
 Book(title='One Hundred Years of Solitude', author='Gabriel García Márquez', year=1967)]


The base model is still optional:

In [6]:
prompts = ["Give the title, author and year of this book separated by commas: " + book for book in test_texts]

results = await processor.process_batch(prompts)
print(results)

['To Kill a Mockingbird, Harper Lee, 1960', 'The Great Gatsby, F. Scott Fitzgerald, 1925', '1984, George Orwell, 1949', 'Pride and Prejudice, Jane Austen, 1813', 'One Hundred Years of Solitude, Gabriel García Márquez, 1967']


# Process Pandas Dataframe

We can process a pandas dataframe by calling the ```process_df``` method. When a response model is provided, the method returns a dataframe with the same columns as the input dataframe, plus the columns specified in the response model.

The ```prompt_column``` argument is the name of the column in the dataframe that contains the text to be processed.

In [7]:
# Test data
test_texts = [
    "To Kill a Mockingbird by Harper Lee (1960)",
    "The Great Gatsby Author: F. Scott Fitzgerald Publication Year: 1925",
    "Book: 1984 Writer: George Orwell Year: 1949",
    "Austen, Jane. Pride and Prejudice. 1813.",
    "One Hundred Years of Solitude - Gabriel García Márquez, published in 1967"
]

df = pd.DataFrame({'id': [1, 2, 3, 4, 5], 'prompts': test_texts})
display(df)

,id,prompts
0,1,To Kill a Mockingbird by Harper Lee (1960)
1,2,The Great Gatsby Author: F. Scott Fitzgerald P...
2,3,Book: 1984 Writer: George Orwell Year: 1949
3,4,"Austen, Jane. Pride and Prejudice. 1813."
4,5,One Hundred Years of Solitude - Gabriel García...


In [9]:
# With response model

class Book(BaseModel):
    title: str
    author: str
    year: int

# The 'prompts' column contains the text to be processed
results = await processor.process_dataframe(df, 'prompts', response_model=Book)
display(results)

,id,prompts,title,author,year
0,1,To Kill a Mockingbird by Harper Lee (1960),To Kill a Mockingbird,Harper Lee,1960
1,2,The Great Gatsby Author: F. Scott Fitzgerald P...,The Great Gatsby,F. Scott Fitzgerald,1925
2,3,Book: 1984 Writer: George Orwell Year: 1949,1984,George Orwell,1949
3,4,"Austen, Jane. Pride and Prejudice. 1813.",Pride and Prejudice,Jane Austen,1813
4,5,One Hundred Years of Solitude - Gabriel García...,One Hundred Years of Solitude,Gabriel García Márquez,1967


We can also use the argument output_format=objects to return a list of objects, including all other columns in the input dataframe.

This is useful for downstream logic, ie. using in an API or database.

In [12]:
class Book(BaseModel):
    title: str
    author: str
    year: int
    
results = await processor.process_dataframe(df, 'prompts', response_model=Book, output_format='objects')
print(type(results[0]), '\n')
display(results)

<class 'utils.llm.batch_llm.EnrichedBook'> 



[EnrichedBook(title='To Kill a Mockingbird', author='Harper Lee', year=1960, id=1, prompts='To Kill a Mockingbird by Harper Lee (1960)'),
 EnrichedBook(title='The Great Gatsby', author='F. Scott Fitzgerald', year=1925, id=2, prompts='The Great Gatsby Author: F. Scott Fitzgerald Publication Year: 1925'),
 EnrichedBook(title='1984', author='George Orwell', year=1949, id=3, prompts='Book: 1984 Writer: George Orwell Year: 1949'),
 EnrichedBook(title='Pride and Prejudice', author='Jane Austen', year=1813, id=4, prompts='Austen, Jane. Pride and Prejudice. 1813.'),
 EnrichedBook(title='One Hundred Years of Solitude', author='Gabriel García Márquez', year=1967, id=5, prompts='One Hundred Years of Solitude - Gabriel García Márquez, published in 1967')]

## Without a response_model

If you choose not to provide a response model, a llm_output column will be appended to the input dataframe.

In [13]:
df['prompts'] = 'Give the title, author and published year of this book separated by a comma: ' + df['prompts']

results = await processor.process_dataframe(df, 'prompts')
display(results)

,id,prompts,llm_output
0,1,"Give the title, author and published year of t...","To Kill a Mockingbird, Harper Lee, 1960"
1,2,"Give the title, author and published year of t...","The Great Gatsby, F. Scott Fitzgerald, 1925"
2,3,"Give the title, author and published year of t...","1984, George Orwell, 1949"
3,4,"Give the title, author and published year of t...","Pride and Prejudice, Jane Austen, 1813"
4,5,"Give the title, author and published year of t...","One Hundred Years of Solitude, Gabriel García ..."


# Daisy Chaining Models

This also makes it easy to see how the the outputs of multiple models compare for a given task.

In [11]:
# Import the gsm8k math benchmark from HuggingFace
splits = {'train': 'main/train-00000-of-00001.parquet', 'test': 'main/test-00000-of-00001.parquet'}
tasks = pd.read_parquet("hf://datasets/openai/gsm8k/" + splits["train"])[:6]


# Set pandas options for better display
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full column content

display(tasks)

,question,answer
0,"Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?",Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72
1,"Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?","Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10"
2,"Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?","In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.\nBetty's grandparents gave her 15 * 2 = $<<15*2=30>>30.\nThis means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.\n#### 5"
3,"Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?","Maila read 12 x 2 = <<12*2=24>>24 pages today.\nSo she was able to read a total of 12 + 24 = <<12+24=36>>36 pages since yesterday.\nThere are 120 - 36 = <<120-36=84>>84 pages left to be read.\nSince she wants to read half of the remaining pages tomorrow, then she should read 84/2 = <<84/2=42>>42 pages.\n#### 42"
4,James writes a 3-page letter to 2 different friends twice a week. How many pages does he write a year?,He writes each friend 3*2=<<3*2=6>>6 pages a week\nSo he writes 6*2=<<6*2=12>>12 pages every week\nThat means he writes 12*52=<<12*52=624>>624 pages a year\n#### 624
5,"Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?","There are 80/100 * 10 = <<80/100*10=8>>8 more purple flowers than yellow flowers.\nSo in Mark's garden, there are 10 + 8 = <<10+8=18>>18 purple flowers.\nPurple and yellow flowers sum up to 10 + 18 = <<10+18=28>>28 flowers.\nThat means in Mark's garden there are 25/100 * 28 = <<25/100*28=7>>7 green flowers.\nSo in total Mark has 28 + 7 = <<28+7=35>>35 plants in his garden.\n#### 35"


In [13]:
instructions = """
    You are a math problem solver. For each problem, show your work step-by-step and use the following format:
    
    1. Write each calculation step clearly
    2. For any arithmetic operation, show it in double angle brackets with the calculation and result: <<calculation=result>>
    3. End your response with "#### [final_answer]" where final_answer is just the numerical result
    
    Example:
    Problem:  James is a first-year student at a University in Chicago. He has a budget of $1000 per semester. He spends 30% of his money on food, 15% on accommodation, 25% on entertainment, and the rest on coursework materials. How much money does he spend on coursework materials?
    Response: Accommodation is 15% * $1000=$<<15*.01*1000=150>>150 Food is 30% * $1000=$<<30*.01*1000=300>>300 Entertainment is 25% * $1000=$<<25*.01*1000=250>>250 Coursework materials are thus $1000-($150+$300+$250) = $300 #### 300
    
    Always show intermediate calculations in the <<calculation=result>> format and conclude with #### followed by the final numerical answer.

    Solve this problem:\n
"""

tasks['question'] = instructions + tasks['question']


In [14]:
# Set up configs for each model we want to compare
config_claude = LLMConfigs.openrouter(model="anthropic/claude-sonnet-4")
config_gemini = LLMConfigs.openrouter(model="google/gemini-2.5-flash-lite")
config_qwen = LLMConfigs.openrouter(model="qwen/qwen3-235b-a22b-2507")

# Set up BatchProcess for each
processor_claude = BatchProcessor(llm_config=config_claude)
processor_qwen = BatchProcessor(llm_config=config_qwen)
processor_gemini = BatchProcessor(llm_config=config_gemini)

# Run each model sequentially
tasks = await processor_claude.process_dataframe(tasks, 
                                                   prompt_column='question', 
                                                   output_column_name='claude')
tasks = await processor_qwen.process_dataframe(tasks, 
                                                   prompt_column='question', 
                                                   output_column_name='qwen')
tasks = await processor_gemini.process_dataframe(tasks, 
                                                   prompt_column='question', 
                                                   output_column_name='gemini')

display(tasks)

,question,answer,claude,qwen,gemini
0,"\n You are a math problem solver. For each problem, show your work step-by-step and use the following format:\n\n 1. Write each calculation step clearly\n 2. For any arithmetic operation, show it in double angle brackets with the calculation and result: <<calculation=result>>\n 3. End your response with ""#### [final_answer]"" where final_answer is just the numerical result\n\n Example:\n Problem: James is a first-year student at a University in Chicago. He has a budget of $1000 per semester. He spends 30% of his money on food, 15% on accommodation, 25% on entertainment, and the rest on coursework materials. How much money does he spend on coursework materials?\n Response: Accommodation is 15% * $1000=$<<15*.01*1000=150>>150 Food is 30% * $1000=$<<30*.01*1000=300>>300 Entertainment is 25% * $1000=$<<25*.01*1000=250>>250 Coursework materials are thus $1000-($150+$300+$250) = $300 #### 300\n\n Always show intermediate calculations in the <<calculation=result>> format and conclude with #### followed by the final numerical answer.\n\n Solve this problem:\n\n\n You are a math problem solver. For each problem, show your work step-by-step and use the following format:\n\n 1. Write each calculation step clearly\n 2. For any arithmetic operation, show it in double angle brackets with the calculation and result: <<calculation=result>>\n 3. End your response with ""#### [final_answer]"" where final_answer is just the numerical result\n\n Example:\n Problem: James is a first-year student at a University in Chicago. He has a budget of $1000 per semester. He spends 30% of his money on food, 15% on accommodation, 25% on entertainment, and the rest on coursework materials. How much money does he spend on coursework materials?\n Response: Accommodation is 15% * $1000=$<<15*.01*1000=150>>150 Food is 30% * $1000=$<<30*.01*1000=300>>300 Entertainment is 25% * $1000=$<<25*.01*1000=250>>250 Coursework materials are thus $1000-($150+$300+$250) = $300 #### 300\n\n Always show intermediate calculations in the <<calculation=result>> format and conclude with #### followed by the final numerical answer.\n\n Solve this problem:\n\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?",Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72,"I need to find the total number of clips Natalia sold in April and May.\n\nFirst, let me identify what I know:\n- In April, Natalia sold clips to 48 friends\n- In May, she sold half as many clips as in April\n\nLet me calculate the number of clips sold in May:\nMay sales = half of April sales = 48 ÷ 2 = <<48/2=24>>24 clips\n\nNow I can find the total clips sold in both months:\nTotal clips = April clips + May clips = 48 + 24 = <<48+24=72>>72 clips\n\n#### 72","Natalia sold 48 clips in April. \nIn May, she sold half as many clips as in April. \n\nClips sold in May = 48 ÷ 2 = <<48/2=24>>24 \n\nTotal clips sold in April and May = 48 + 24 = <<48+24=72>>72 \n\n#### 72","Natalia sold 48 clips in April.\nIn May, she sold half as many clips as in April.\n\nNumber of clips sold in May = <<48/2=24>>24\n\nTotal number of clips sold altogether in April and May = Number of clips sold in April + Number of clips sold in May.\nTotal clips sold = 48 + 24\n\nTotal clips sold = <<48+24=72>>72\n#### 72"
1,"\n You are a math problem solver. For each problem, show your work step-by-step and use the following format:\n\n 1. Write each calculation step clearly\n 2. For any arithmetic operation, show it in double angle brackets with the calculation and result: <<calculation=result>>\n 3. End your response with ""#### [final_answer]"" where final_answer is just the numerical result\n\n Example:\n Problem: James is a first-year student at a University in Chicago. He has a budget of $1000 per semester. He spends 30% of his money on food